# Refractive Indices of Glasses

**Scott Prahl**

**Sept 2023**

The interface is a bit weird.  Basically, you need to get the Sellmeier coefficients for the type of glass that you want.  Fortunately, this module has a bunch of glasses entered already.  The list is named `ALL_GLASS_NAMES`.

In [ ]:
%config InlineBackend.figure_format='retina'

import sys
import numpy as np
import matplotlib.pyplot as plt

if sys.platform == "emscripten":
    import micropip
    await micropip.install("ofiber")

import ofiber

In [ ]:
print(ofiber.ALL_GLASS_NAMES)

You can just count to find the glass you want.  Standard silicon dioxide SiO₂ has number 0.  Germanium dioxide GeO₂ has number 1.

Alternatively you can search for the glass you want.  The number of BK7 is 13.

In [ ]:
ofiber.find_glass("BK7")

So `ofiber.glass(13)` will return the coefficients for BK7 glass for example.  The function `ofiber.n(glass,λ)` returns the index of refraction at a particular (vacuum) wavelength $\lambda_0$.

In [ ]:
glass_coefficients = ofiber.glass(13)
name = ofiber.glass_name(13)
λ = np.linspace(1000, 1700, 100) * 1e-9
n = ofiber.n(glass_coefficients, λ)
plt.figure(figsize=(8, 4.5))
plt.plot(λ * 1e9, n)
plt.title(name)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Index of Refraction")
plt.show()

## SiO₂ and GeO₂ fibers

These are the mainstay of the optical fiber industry.  By doping SiO₂ with GeO₂ different refractive index profiles can be achieved.  This plot shows the limits of 100% of each material.  They look pretty similar, just displaced by 0.14 index units.

In [ ]:
glass = ofiber.glass(0)
name = ofiber.glass_name(0)
λ = np.linspace(1000, 1700, 100) * 1e-9
n = ofiber.n(glass, λ)
plt.figure(figsize=(8, 4.5))
plt.plot(λ * 1e9, n, label=name)

glass = ofiber.glass(1)
name = ofiber.glass_name(1)
λ = np.linspace(1000, 1700, 100) * 1e-9
n = ofiber.n(glass, λ)
plt.plot(λ * 1e9, n, label=name)

plt.title("SiO₂ and GeO₂ ")
plt.xlabel("Wavelength (nm)")
plt.ylabel(r"$n(\lambda)$")
plt.legend()
plt.show()

However, if you plot the first derivative, interesting things appear.  Notice the weird units on the vertical axis.

In [ ]:
glass = ofiber.glass(0)
name = ofiber.glass_name(0)
λ = np.linspace(1000, 1700, 100) * 1e-9
dn = ofiber.dn(glass, λ)
plt.figure(figsize=(8, 4.5))
plt.plot(λ * 1e9, dn * 1e-6, label=name)

glass = ofiber.glass(1)
name = ofiber.glass_name(1)
λ = np.linspace(1000, 1700, 100) * 1e-9
dn = ofiber.dn(glass, λ)
plt.plot(λ * 1e9, dn * 1e-6, label=name)

plt.title("SiO₂ and GeO₂ ")
plt.xlabel("Wavelength (nm)")
plt.ylabel(r"dn/d$\lambda$ [1/micron]")
plt.legend()
plt.show()

## SiO₂ doped with GeO₂ 

Since doping of SiO₂ with GeO₂ is common there is a special routine to generate Sellmeier coefficients for glass doped with a specific fraction $x$ of GeO₂.  The interface is simple.

In [ ]:
help(ofiber.doped_glass)

Here the second derivative of the refractive index is plotted for 100% SiO₂, 100% GeO₂, and 10% GeO₂

In [ ]:
glass = ofiber.glass(0)
name = ofiber.glass_name(0)
λ = np.linspace(1000, 1700, 100) * 1e-9
d2n = ofiber.d2n(glass, λ)
plt.figure(figsize=(8, 4.5))
plt.plot(λ * 1e9, d2n * 1e-12, label=name)

glass = ofiber.glass(1)
name = ofiber.glass_name(1)
λ = np.linspace(1000, 1700, 100) * 1e-9
d2n = ofiber.d2n(glass, λ)
plt.plot(λ * 1e9, d2n * 1e-12, label=name)

glass = ofiber.doped_glass(0.10)
name = ofiber.doped_glass_name(0.1)
λ = np.linspace(1000, 1700, 100) * 1e-9
d2n = ofiber.d2n(glass, λ)
plt.plot(λ * 1e9, d2n * 1e-12, label=name)

plt.plot([1000, 1700], [0, 0], ":k")
plt.title("SiO$_2$ and GeO$_2$")
plt.xlabel("Wavelength (nm)")
plt.ylabel(r"$d^2n/d\lambda^2$    [1/$\mu$m$^2$]")
plt.legend()
plt.show()

If we just want the second derivative at a specific wavelength, that is simple too.  The only tricky part is converting from the default 1/m$^2$ to 1/µm$^2$

In [ ]:
λ = 1550e-9
glass = ofiber.glass(0)
name = ofiber.glass_name(0)
disp = ofiber.d2n(glass, λ)
print("d^2n/dlambda^2 of %s at %.0f nm is %.4f/um**2" % (name, λ * 1e9, disp * 1e-12))

We use this to calculate the material, waveguide, and total dispersion for a fiber.  (These are available directly in the `ofiber.dispersion` module.  This is just an example.)

In [ ]:
# Ghatak Section 3.A.1, page 85.
#
c = 3e8  # [m/s]

a = 4.1e-6  # [m] Fiber radius
delta = 0.0027  # Fractional change in the index of refraction

start = 1100
finish = 1700
resolution = 10

λ = np.arange(start, finish, resolution) * 1e-9
npoints = len(λ)

glass = ofiber.doped_glass(0)
n1 = ofiber.n(glass, λ)
d2n = ofiber.d2n(glass, λ)

V = (2 * np.pi / λ) * a * n1 * np.sqrt(2 * delta)

# Material Dispersion
M = -(λ / c) * d2n

# Waveguide dispersion
Dw = -n1 * (1 + delta) * delta / c / λ * (0.080 + 0.549 * (2.834 - V) ** 2)

ps_nm_km = 1e-12 / 1e-9 / 1e3
plt.subplots(1, 1, figsize=(8, 4.5))
plt.plot(λ * 1e9, M / ps_nm_km, color="blue")

plt.plot(λ * 1e9, Dw / ps_nm_km, color="orange")
plt.plot(λ * 1e9, (M + Dw) / ps_nm_km, color="red")
plt.axhline(0, color="black", linewidth=0.5)
plt.xlabel("Wavelength [microns]")
plt.ylabel("Fiber Dispersion [ps/km/nm]")
plt.title(r"SiO₂ core, Diameter=%.1f$\mu$m, Δ=%.4f" % (a * 2e6, delta))
plt.xlim(1100, 1700)
plt.text(1510, 10, "Total Dispersion", color="red")
plt.text(1600, 26, "Material Dispersion", color="blue", ha="right")
plt.text(1400, -10, "Waveguide Dispersion", color="orange")
plt.axvline(1315, color="black", linewidth=0.5)
plt.text(1300, 10, " zero dispersion λ", rotation=90)
# plt.savefig('dispersion.svg')
plt.show()